# 瞎扯机器人之语种不限  WeChaty + PaddleHub

## 一、项目背景
基于百度翻译API， PaddleHub 以及 WeChaty实现微信端不限语种的聊天机器人。聊天时，你说啥语言就回复你啥语言，并保持聊天环境上下文。目前是首个使用微信客户端交互的多轮对话模型。

本项目是参加【**AI ChatBot 创意赛**】的作品，PaddleHub携手开源聊天机器人框架WeChaty带来 AI ChatBot创意赛，为AI算法工程师提供一个全新的应用场景：Chatbot （Conversational AI），同时也为Chatbot 开发者提供一个全新的AI能力平台，拓宽视野，为未来设计更加强大的 Chatbot 提供一扇门。

本项目主要难点是协调不同语言内容在多轮对话中进行交互，展现plato2模型强大的能力。另一方面亮点，结合了百度翻译开放API，无缝对接，最终通过Wechaty保证了用户在终端丝滑般的切换语言聊天。用户可以很方便的体验聊天的乐趣，寻找聊天话题不冷场，还能将本应用当成外语学习的好工具。

<iframe style="width:98%;height: 450px;" src="//player.bilibili.com/player.html?aid=930461629&bvid=BV1sK4y1d7ys&cid=334869201&page=1" scrolling="no" border="0" frameborder="no" framespacing="0" allowfullscreen="true"> </iframe>


**b站链接:[https://www.bilibili.com/video/BV1sK4y1d7ys/](https://www.bilibili.com/video/BV1sK4y1d7ys/)**

## 二、安装环境

```
import paddle.fluid##检查飞桨安装情况
paddle.fluid.install_check.run_check()

!pip install paddlehub==1.8.2 --upgrade -i https://pypi.tuna.tsinghua.edu.cn/simple  # 安装最新版本，使用清华源
!hub install plato2_en_base==1.0.0
```

## 三、关于WeChaty和PaddleHub

wechaty（https://github.com/wechaty/wechaty）是一款开源的微信SDK，它基于微信公开的API，对接口进行了一系列的封装，提供一系列简单的接口，然后开发者可以在其之上进行微信机器人的开发。

PaddleHub 是基于 PaddlePaddle 开发的预训练模型管理工具，可以借助预训练模型更便捷地开展迁移学习工作，旨在让 PaddlePaddle 生态下的开发者更便捷体验到大规模预训练模型的价值。

关于二者的结合使用请参考我的另一个公开项目：
	[微信医聊自动问答 WeChaty + PaddleHub](https://aistudio.baidu.com/aistudio/projectdetail/1868162)




这里使用docker脚本可方便快速部署。请将your_token处替换成你的WeChaty token (**puppet_padlocal_xxxxxxxxxxxxx)**

```
export WECHATY_LOG="verbose"
export WECHATY_PUPPET="wechaty-puppet-padlocal"
export WECHATY_PUPPET_PADLOCAL_TOKEN="your_token"

export WECHATY_PUPPET_SERVER_PORT="8080"
export WECHATY_TOKEN="your_token"

docker run -ti \
  --name wechaty_puppet_service_token_gateway \
  --rm \
  -e WECHATY_LOG \
  -e WECHATY_PUPPET \
  -e WECHATY_PUPPET_PADLOCAL_TOKEN \
  -e WECHATY_PUPPET_SERVER_PORT \
  -e WECHATY_TOKEN \
  -p "$WECHATY_PUPPET_SERVER_PORT:$WECHATY_PUPPET_SERVER_PORT" \
  wechaty/wechaty:latest
~                        
```

运行成功后如下图:


![](https://ai-studio-static-online.cdn.bcebos.com/73f123bb9b37410eb49f834dc512226e014f3f53a77d42888b201833e8ef6942)

## 三、关于plato2

PLATO2是一个超大规模生成式对话系统模型。它承袭了PLATO隐变量进行回复多样化生成的特性，能够就开放域话题进行流畅深入的聊天。在paddlehub中已经集成了两个plato对话模型，分别是plato2_en_base和plato2_en_large。本文使用的是小模型plato2_en_base， 相比plato2_en_large会弱一些，本文使用的模型是plato2_en_base，有兴趣的同学可以自己尝试下plato2_en_large。

参考大佬的项目：[“没有对象就自己造”系列之 用paddlehub实现多轮对话](https://aistudio.baidu.com/aistudio/projectdetail/1197592)
因考虑到训练时长和数据集，本相关目主要是为了体验，所以没像大佬(没入门的研究生)那样进行训练测试。

 论文：[PLATO-2: Towards Building an Open-Domain Chatbot via Curriculum Learning](https://arxiv.org/abs/2006.16779)
 
 代码原址：[github](https://github.com/PaddlePaddle/Knover/blob/master/plato-2)


Plato是百度推出的一个开放域对话系统，经过打磨，如今已经放出了Plato-2版本，不过可能出于商业化考虑，并没有放出很详细的教程，也没有放出中文的预训练模型，如果个人训练的话可能会有很大的困难（论文中用64块V100训练了3周才得到的模型）。

![](https://ai-studio-static-online.cdn.bcebos.com/dd01edc2b65140058209d394a78564622e47f6b9cc0f4985a61834658fb70173)


相较于其他开放域对话系统，Plato的优势在于一问多答情景下的处理，即一个问题对应多个不同的答复，找出合理的答复进行对话。这个模型包括两个学习过程（如上图），第一阶段用当前表现很好的一对一对话的模型进行初步训练；第二步进行微调，学习不同情景的对话生成并进行评估。由于Plato采用了更好的策略应付更加现实的场景，其在对话评估取得了目前最好的结果。

常用的参数:

```
train.py
--train_file 训练文件地址

--valid_file 评估文件地址

--model 用到的模型名称：Plato：plato；NSPModel：next_sentence_prediction model

--config_path 模型细节参数配置文件，如24L.json

--task 模型应用任务 NextSentencePrediction；DialogGeneration；UnifiedTransformer

--vocab_path 词集路径

--spm_model_file sentencepiece model文件的路径

--num_epochs 训练周期数

--log_steps 输出训练详情的间隔步数

--validation_steps 评价间隔步数

--save_steps 保存间隔步数

infer.py
--infer_file 需要推断的文件

--output_name 需要保存的对象，response；data_id；score

--model 用到的模型名称：Plato：plato；NSPModel：next_sentence_prediction model

--config_path 模型细节参数配置文件，如24L.json

--task 模型应用任务 NextSentencePrediction；DialogGeneration；UnifiedTransformer

--vocab_path 词集路径

--spm_model_file sentencepiece model文件的路径
```

## 四、关于百度翻译API

PaddleNLP中已有三款性能不错的机器翻译模型, 分别是(seq2seq, text_simultaneous_translation, transformer)， 但是都需要大量的数据和训练资源， 集成在项目里复杂度也较高，不太适合这篇新手体验项目。而且经过测试，百度翻译API的效果要远比自己训练的要好，响应和API设计都能达到要求。

使用[百度翻译API](https://api.fanyi.baidu.com/)就极其简单了， 就需要简单的二三十行代码，就能实现近实时的文本翻译以及多语种交互翻译。

准备工作主要是两步：

**第一步：**
注册百度账号（自行注册）

**第二步**
申请百度翻译api ，获得appid以及securityKey
申请教程：https://jingyan.baidu.com/article/3f16e00305bb552591c10304.html

**第三步**
下载相应的示例代码，修改appid和securityKey就可以跑通了。 可参考我的代码： baidu_translate.py

![](https://ai-studio-static-online.cdn.bcebos.com/a7fcd3d03fa0435d86eed32dc74c8d8d5431d8da2f554a7f9e7a5e009ecd347c)





## 五、运行Plato2和百度翻译

总体上效果还行, 虽然上下文环境的保持还是不够优秀，语言翻译额时候也会有一些偏差，但是已经能满足瞎扯的需求了， 瞎扯的时候还能练练外语。
**注意：要在GPU环境下运行**



In [1]:
import paddlehub as hub
import os
from baidu_translate import translate

os.environ["CUDA_VISIBLE_DEVICES"] = '0'
module = hub.Module('plato2_en_base')

with module.interactive_mode(max_turn=20):
    while True:
        human_utterance = input()
        if len(human_utterance) > 0:
            print("输入文本: " + human_utterance) #以中文, 德语, 英语为例
            lan, en_txt = translate(human_utterance)

            if lan == "en":
                en_txt = human_utterance
            
            robot_utterance = module.generate(human_utterance)

            if lan == "en":
                print("Robot: %s" % robot_utterance[0])
                continue
            else:
                _, src_txt = translate(robot_utterance[0], "en", lan)
                print("Robot: %s" % src_txt)
                continue

        else:
            break

[2021-05-02 00:50:01,181] [    INFO] - Installing plato2_en_base module
[2021-05-02 00:50:01,281] [    INFO] - Module plato2_en_base already installed in /home/aistudio/.paddlehub/modules/plato2_en_base


Load pretraining parameters from /home/aistudio/.paddlehub/modules/plato2_en_base/assets/24L/Plato.
输入文本: Hello, are you there?
Robot: Hello I am . Where are you from ?
输入文本: Ich komme aus China
Robot: Das ist cool.Wo in China?Wenn ich fragen darf...Welchen Teil?Ich bin aus Hongkong, also kenne ich das Gefühl:)
输入文本: Ich bin von Beijing
Robot: I know Beijing is a very large city . Why did you decide to live there ?
输入文本: 北京是个很好的城市
Robot: 是什么让你选择了北京？我听说它真的很漂亮
输入文本: for money
Robot: What do you mean ? Do you work in Beijing ? And also , where did you live before ?
输入文本: Ich wohne in Berlin zuvor.
Robot: Oh, cool.Du bist einer der wenigen, die in der Nähe von Peking leben.Wie ist Berlin?
输入文本: leider zu Kalt hier
Robot: Haha, Ich werde auch eine Berliner in Peking gestalten.


## 六、部署在wechaty和脚本任务上

实测效果还不错~~~

![](https://ai-studio-static-online.cdn.bcebos.com/02ee12279f984b88bb1653b8c78976219f8daf4b609c4bf69e3deb7f7a62472b)

![](https://ai-studio-static-online.cdn.bcebos.com/2cb8ac3e20c94a89a434ebf05eafec8532a452cb709d4381864f8ea0aa3485c0)




## 七、总结

本项目尝试了plato2， 并结合百度AI开放的机器翻译API，打造了一个不限语种的聊天机器人，经过测试效果还挺不错。

后续我将手动训练机器翻译模型以及plato2对话模型。

**关于作者**

PaddlePaddle开发爱好者

我在AI Studio上获得黄金等级，点亮5个徽章，来互关呀~ https://aistudio.baidu.com/aistudio/personalcenter/thirdview/89442

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 